# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

M. Sharma  ->  Y. M. Sharma  |  ['M. Sharma']
A. Dimoff  ->  A. Dimoff  |  ['A. Dimoff']
R. Seeburger  ->  L. R. Seeburger  |  ['R. Seeburger']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']


Arxiv has 66 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2505.21630


extracting tarball to tmp_2505.21630... done.


Retrieving document from  https://arxiv.org/e-print/2505.22201


extracting tarball to tmp_2505.22201...

 done.


A. Dimoff  ->  A. Dimoff  |  ['A. Dimoff']


Found 90 bibliographic references in tmp_2505.22201/Modeling_the_progenitors_of_low_mass_post_accretion_binaries.bbl.
Issues with the citations
syntax error in line 144: '=' expected
Retrieving document from  https://arxiv.org/e-print/2505.22369


extracting tarball to tmp_2505.22369...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2505.22201-b31b1b.svg)](https://arxiv.org/abs/2505.22201) | **Modeling the progenitors of low-mass post-accretion binaries**  |
|| <mark>A. Dimoff</mark>, et al. -- incl., <mark>R. Seeburger</mark> |
|*Appeared on*| *2025-05-29*|
|*Comments*| **|
|**Abstract**|            About half of the mass of all heavy elements with mass number A > 90 is formed through the slow neutron capture process (s-process), occurring in evolved asymptotic giant branch (AGB) stars with masses ~1-6 $\rm{M_{\odot}}$. The s-process can be studied by modeling the accretion of material from AGB stars onto binary barium (Ba), CH, and carbon-enhanced metal-poor (CEMP)-s stars. Comparing observationally derived surface parameters and 1D-LTE abundance patterns of s-process elements to theoretical binary accretion models, we aim to understand the formation of post-accretion systems. We explore the extent of dilution of the accreted material and describe the impact of convective mixing on the observed surface abundances. We compute a new grid of 2700 accretion models for low-mass post-accretion systems. A maximum-likelihood comparison determines the best fit models for observational samples of Ba, CH, and CEMP-s stars. We find consistent AGB donor masses in the mass range of 2-3 $\rm{M_{\odot}}$ across our sample of post-accretion binaries. We find the formation scenario for weak Ba stars is an AGB star transferring a moderate amount of mass ($\leq$0.5 $\rm{M_{\odot}}$) resulting in a ~2.0-2.5 $\rm{M_{\odot}}$ star. The strong Ba stars are best fit with lower final masses ~1.0-2.0 $\rm{M_{\odot}}$, and significant accreted mass ($\geq$0.5 $\rm{M_{\odot}}$). The CH and CEMP-s stars display lower final masses (~1.0 $\rm{M_{\odot}}$) and small amounts of transferred material (~0.1 $\rm{M_{\odot}}$). We find that Ba stars generally accrete more material than CEMP-s and CH stars. We also find that strong Ba stars must accrete more than 0.50 $\rm{M_{\odot}}$ to explain their abundance patterns, and in this limit we are unable to reproduce the observed mass distribution of strong Ba stars. The mass distributions of the weak Ba stars, CEMP-s, and CH stars are well reproduced in our modeling.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2505.21630-b31b1b.svg)](https://arxiv.org/abs/2505.21630) | **Quasar absorption outflows on galactic scales: Insights from DESI**  |
|| M. Dehghanian, et al. -- incl., <mark>M. Sharma</mark> |
|*Appeared on*| *2025-05-29*|
|*Comments*| *8 pages, 5 figures-Accepted for publications in MNRAS*|
|**Abstract**|            Absorption outflows in quasars play an important role in understanding active galactic nuclei (AGN) feedback and their influence on galaxy evolution. The unprecedented spectral data provided by the Dark Energy Spectroscopic Instrument (DESI) opens new avenues to explore these outflows. We analyze five low-ionization absorption outflow systems in four intermediate-redshift quasars (2 < z < 3) using the data obtained by DESI in order to characterize their physical properties and energetics, and also to assess their role in AGN feedback. We use the spectra from DESI's Early Data Release to determine the ionic column densities, total hydrogen column densities, electron number densities, and ionization parameters via photoionization modeling and absorption line analysis. We derive the outflows' distance from the AGN, and its kinematic properties: mass-flow rates, kinetic luminosity, and momentum flux. Our study identifies five distinct mini-broad absorption line outflow systems, hosted by four quasars. The identified outflows exhibit hydrogen column densities of \log (N_H) = 20.0-20.7[cm^{-2}], ionization parameters of log (U_H) = -2.7 to -2.1, and electron number densities of log (n_e) =1.45-2.85[cm^{-2}]. The distances of the outflows from the central source are between 4.5 to 31 kpc, and the kinetic luminosities range from 2\times10^{-6} to 5\times10^{-3} of the Eddington luminosity. The outflows in J1407 and J1032 show the largest kinetic luminosities, with momentum flux ratios (\dot{p} / \dot{p}_{rad}) of about 2 and 0.2, respectively. Our findings highlight the vital role of DESI data in uncovering the diversity and significance of quasar outflows in galaxy evolution.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2505.22369-b31b1b.svg)](https://arxiv.org/abs/2505.22369) | **Model-independent cosmological inference after the DESI DR2 data with improved inverse distance ladder**  |
|| J.-L. Ling, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-05-29*|
|*Comments*| *13 pages, 4 figures*|
|**Abstract**|            Recently, the baryon acoustic oscillations (BAO) measurements from the DESI survey have suggested hints of dynamical dark energy, challenging the standard $\Lambda $CDM model. In this work, we adopt an improved inverse distance ladder approach based on the latest cosmological data to provide a model-independent perspective, employing a global parametrization based on cosmic age (PAge). Our analysis incorporates DESI DR2 BAO measurements, cosmic chronometer (CC) data, and type Ia supernovae (SNe) observations from either the DESY5 or PantheonPlus datasets. For the DESY5+DESI DR2+CC datasets, we obtain $H_0 = 67.91 \pm 2.33~\mathrm{km~s^{-1}~Mpc^{-1}}$. This value is consistent with the Planck 2018 result, while shows $2.0 \sigma$ tension with the SH0ES measurement. Furthermore, by mapping specific cosmological models into PAge approximation parameter space $(p_{\mathrm{age}}, \eta)$, our model-independent analysis reveals a notable deviation from the $\Lambda \mathrm{CDM}$ model, as indicated by the DESY5 and DESI DR2 datasets. Finally, DESY5+DESI DR2+CC datasets provide nearly decisive evidence favoring the PAge model over the standard $\Lambda \mathrm{CDM}$ model. These findings highlight the need for further investigation into the expansion history to better understand the deviations from the $\Lambda \mathrm{CDM}$ model.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2505.22201/./figures/Ba_star_accretion_phase_notitle.png', 'tmp_2505.22201/./figures/obs_samples_kiel.png', 'tmp_2505.22201/./figures/Kipp_2p0_2p5_2p5_z010.png']
copying  tmp_2505.22201/./figures/Ba_star_accretion_phase_notitle.png to _build/html/
copying  tmp_2505.22201/./figures/obs_samples_kiel.png to _build/html/
copying  tmp_2505.22201/./figures/Kipp_2p0_2p5_2p5_z010.png to _build/html/
exported in  _build/html/2505.22201.md
    + _build/html/tmp_2505.22201/./figures/Ba_star_accretion_phase_notitle.png
    + _build/html/tmp_2505.22201/./figures/obs_samples_kiel.png
    + _build/html/tmp_2505.22201/./figures/Kipp_2p0_2p5_2p5_z010.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\CJH}[1]{\textcolor{brightpink}{{\bf} #1}}$
$\newcommand{\AJD}[1]{\textcolor{electricblue}{#1}}$
$\newcommand{\isotope}[2]{^{#1}{#2}}$
$\newcommand{\msun}{\rm{M_{\odot}}}$</div>



<div id="title">

# Modeling the progenitors of low-mass post-accretion binaries

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2505.22201-b31b1b.svg)](https://arxiv.org/abs/2505.22201)<mark>Appeared on: 2025-05-29</mark> - 

</div>
<div id="authors">

<mark>A. Dimoff</mark>, et al.

</div>
<div id="abstract">

**Abstract:** About half of the mass of all heavy elements with mass number A $>$ 90 is formed through the slow neutron capture process (s-process), occurring in evolved asymptotic giant branch (AGB) stars with masses $\sim1-6$ $\msun$ . The s-process can be studied by modeling the accretion of material from AGB stars onto binary barium (Ba), CH, and carbon-enhanced metal-poor (CEMP)-s stars. Comparing observationally derived surface parameters and 1D-LTE abundance patterns of s-process elements to theoretical binary accretion models, we aim to understand the formation of post-accretion systems. We explore the extent of dilution of the accreted material and describe the impact of convective mixing on the observed surface abundances. We compute a new grid of 2700 accretion models for low-mass post-accretion systems. A maximum-likelihood comparison determines the best fit models for observational samples of Ba, CH, and CEMP-s stars. We find consistent AGB donor masses in the mass range of $2 - 3$ $\msun$ $ $ across our sample of post-accretion binaries. We find the formation scenario for weak Ba stars is an AGB star transferring a moderate amount of mass ( $\leq 0.5$ $\msun$ ) resulting in a $\sim 2.0-2.5$ $\msun$ $ $ Ba star. The strong Ba stars are best fit with lower final masses $\sim1.0-2.0$ $\msun$ $ $ and significant accreted mass ( $\geq 0.5$ $\msun$ ). The CH and CEMP-s stars display lower final masses ( $\sim1.0$ $\msun$ ) and small amounts of transferred material ( $\sim 0.1$ $\msun$ ). We find that Ba stars generally accrete more material than CEMP-s and CH stars. We also find that strong Ba stars must accrete more than $0.50$ $\msun$ $ $ to explain their abundance patterns, and in this limit we are unable to reproduce the observed mass distribution of strong Ba stars. The mass distributions of the weak Ba stars, CEMP-s, and CH stars are well reproduced in our modeling.

</div>

<div id="div_fig1">

<img src="tmp_2505.22201/./figures/Ba_star_accretion_phase_notitle.png" alt="Fig5" width="100%"/>

**Figure 5. -** Left: Evolutionary tracks for a sample of stars with $m_{final} = 2.50$\msun$ $at [Fe/H] = -0.15 with different accretion masses and initial mass ratios, with accretion phases for each model highlighted in blue. Right: Relative surface abundance of the s-process element Ba. The abundance is elevated after the accretion phase, and only after the onset of first dredge-up and mixing is the surface abundance diluted. (*fig:acc_phase*)

</div>
<div id="div_fig2">

<img src="tmp_2505.22201/./figures/obs_samples_kiel.png" alt="Fig1" width="100%"/>

**Figure 1. -** Kiel diagram showing surface gravities and effective temperatures for the collected observational sample. Blue data points are strong Ba stars, cyan data points are weak Ba stars, orange data points are CH stars, and red data points are CEMP-s stars. Surface parameters and abundances are collected from \citet{Dimoff+2024}, \citet{DeCastro_etal2016}, \citet{Roriz2021}, \cite{Goswami_etal2006}, \citet{Karinkuzi_Goswami2014}, \citet{Karinkuzhi_Goswami2015}, \citet{Goswami_Aoki_Karinkuzhi2016}, \citet{Goswami+2021_CEMPs}. (*fig:obs_kiel*)

</div>
<div id="div_fig3">

<img src="tmp_2505.22201/./figures/Kipp_2p0_2p5_2p5_z010.png" alt="Fig2" width="100%"/>

**Figure 2. -** Example Kippenhahn diagram for a 2.00 $\msun$$ $star accreting 0.50 $\msun$$ $of material from a 2.50 $\msun$$ $AGB star at a metallicity of [Fe/H] = -0.15. Green colors denote convective regions, and purple colors denote radiative regions, determined by the computed difference in the radiative and adiabatic transfer gradients. In the total mass on the y-axis, the accretion phase can be identified by the increase in mass. Note that the x-axis is model number, which is non-linear with respect to time. (*fig:Kippenhahn*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2505.22201"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

505  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

12  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

4  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
